In [182]:
import pandas as pd
import utils.DataEngine as de
import matplotlib.pyplot as plt
import collections
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from sklearn.linear_model import Ridge
import numpy as np

In [2]:
de = de.DataEngine('data2024')

In [3]:
df = de.data['round_1']['price_df']

In [4]:
df

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-2,0,AMETHYSTS,10002,1,9996.0,2.0,9995.0,29.0,10004,2,10005.0,29.0,NaN,NaN,10003.0,0.0
1,-2,0,STARFRUIT,5002,1,4997.0,31.0,NaN,NaN,5003,31,NaN,NaN,NaN,NaN,5002.5,0.0
2,-2,100,AMETHYSTS,9996,2,9995.0,22.0,NaN,NaN,10004,2,10005.0,22.0,NaN,NaN,10000.0,0.0
3,-2,100,STARFRUIT,4997,24,NaN,NaN,NaN,NaN,5003,2,5004.0,22.0,NaN,NaN,5000.0,0.0
4,-2,200,STARFRUIT,4997,2,4996.0,20.0,NaN,NaN,5003,20,NaN,NaN,NaN,NaN,5000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-2,199700,AMETHYSTS,9998,8,9996.0,2.0,9995.0,25.0,10004,2,10005.0,25.0,NaN,NaN,10001.0,0.0
3996,-2,199800,STARFRUIT,4949,21,NaN,NaN,NaN,NaN,4956,21,NaN,NaN,NaN,NaN,4952.5,0.0
3997,-2,199800,AMETHYSTS,9995,21,NaN,NaN,NaN,NaN,10002,8,10005.0,21.0,NaN,NaN,9998.5,0.0
3998,-2,199900,STARFRUIT,4950,32,NaN,NaN,NaN,NaN,4954,7,4956.0,32.0,NaN,NaN,4952.0,0.0


In [5]:
sdf = df[df['product'] == 'STARFRUIT'].copy()
adf = df[df['product'] == 'AMETHYSTS'].copy()

In [6]:
adf

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-2,0,AMETHYSTS,10002,1,9996.0,2.0,9995.0,29.0,10004,2,10005.0,29.0,NaN,NaN,10003.0,0.0
2,-2,100,AMETHYSTS,9996,2,9995.0,22.0,NaN,NaN,10004,2,10005.0,22.0,NaN,NaN,10000.0,0.0
5,-2,200,AMETHYSTS,9995,20,NaN,NaN,NaN,NaN,10005,20,NaN,NaN,NaN,NaN,10000.0,0.0
6,-2,300,AMETHYSTS,9996,2,9995.0,29.0,NaN,NaN,10004,2,10005.0,29.0,NaN,NaN,10000.0,0.0
8,-2,400,AMETHYSTS,9996,2,9995.0,25.0,NaN,NaN,10004,2,10005.0,25.0,NaN,NaN,10000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,-2,199500,AMETHYSTS,10002,1,9996.0,1.0,9995.0,22.0,10004,1,10005.0,22.0,NaN,NaN,10003.0,0.0
3992,-2,199600,AMETHYSTS,9996,1,9995.0,21.0,NaN,NaN,10002,7,10004.0,1.0,10005.0,21.0,9999.0,0.0
3995,-2,199700,AMETHYSTS,9998,8,9996.0,2.0,9995.0,25.0,10004,2,10005.0,25.0,NaN,NaN,10001.0,0.0
3997,-2,199800,AMETHYSTS,9995,21,NaN,NaN,NaN,NaN,10002,8,10005.0,21.0,NaN,NaN,9998.5,0.0


In [7]:
sdf

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
1,-2,0,STARFRUIT,5002,1,4997.0,31.0,NaN,NaN,5003,31,NaN,NaN,NaN,NaN,5002.5,0.0
3,-2,100,STARFRUIT,4997,24,NaN,NaN,NaN,NaN,5003,2,5004.0,22.0,NaN,NaN,5000.0,0.0
4,-2,200,STARFRUIT,4997,2,4996.0,20.0,NaN,NaN,5003,20,NaN,NaN,NaN,NaN,5000.0,0.0
7,-2,300,STARFRUIT,4996,31,NaN,NaN,NaN,NaN,5003,31,NaN,NaN,NaN,NaN,4999.5,0.0
9,-2,400,STARFRUIT,4995,27,NaN,NaN,NaN,NaN,5002,27,NaN,NaN,NaN,NaN,4998.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3990,-2,199500,STARFRUIT,4949,23,NaN,NaN,NaN,NaN,4956,23,NaN,NaN,NaN,NaN,4952.5,0.0
3993,-2,199600,STARFRUIT,4949,22,NaN,NaN,NaN,NaN,4956,22,NaN,NaN,NaN,NaN,4952.5,0.0
3994,-2,199700,STARFRUIT,4949,27,NaN,NaN,NaN,NaN,4956,27,NaN,NaN,NaN,NaN,4952.5,0.0
3996,-2,199800,STARFRUIT,4949,21,NaN,NaN,NaN,NaN,4956,21,NaN,NaN,NaN,NaN,4952.5,0.0


In [260]:
LEN = 4
X = pd.DataFrame(columns=[f"lagg {i+1}" for i in range(LEN)])
for i in range(LEN):
    X[f"lagg {i+1}"] = sdf.mid_price.shift(i+1)
X = sm.add_constant(X)
y = sdf.mid_price.iloc[4:]
X = X.iloc[4:]

In [261]:
X

,const,lagg 1,lagg 2,lagg 3,lagg 4
9,1.0,4999.5,5000.0,5000.0,5002.5
11,1.0,4998.5,4999.5,5000.0,5000.0
12,1.0,4999.5,4998.5,4999.5,5000.0
15,1.0,5001.0,4999.5,4998.5,4999.5
17,1.0,5000.0,5001.0,4999.5,4998.5
...,...,...,...,...,...
3990,1.0,4949.5,4951.5,4954.0,4950.5
3993,1.0,4952.5,4949.5,4951.5,4954.0
3994,1.0,4952.5,4952.5,4949.5,4951.5
3996,1.0,4952.5,4952.5,4952.5,4949.5


In [262]:
y

9       4998.5
11      4999.5
12      5001.0
15      5000.0
17      5000.0
         ...  
3990    4952.5
3993    4952.5
3994    4952.5
3996    4952.5
3998    4952.0
Name: mid_price, Length: 1996, dtype: float64

In [263]:
m = sm.OLS(y, X)
res = m.fit()

In [264]:
res.params

const     4.392047
lagg 1    0.330427
lagg 2    0.223498
lagg 3    0.251663
lagg 4    0.193519
dtype: float64

In [265]:
model = AutoReg(sdf['mid_price'], lags=4)
model_fitted = model.fit()
model_fitted.summary()

c:\Users\giova\Git Repositories\Prosperity\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:              mid_price   No. Observations:                 2000
Model:                     AutoReg(4)   Log Likelihood               -3467.787
Method:               Conditional MLE   S.D. of innovations              1.375
Date:                Wed, 27 Mar 2024   AIC                           6947.574
Time:                        21:00:42   BIC                           6981.167
Sample:                             4   HQIC                          6959.910
                                 2000                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            4.3920     10.797      0.407      0.684     -16.769      25.553
mid_price.L1     0.3304      0.022     15.050      0.000       0.287       0.373
mid_price.L2     0.2235      0.022      9.943      0.000       0.179       0.268
mid_price.L3     0.2517      0.022     11.196      0.000       0.208       0.296
mid_price.L4     0.1935      0.022      8.816      0.000       0.150       0.237
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.0004           -0.0000j            1.0004           -0.0000
AR.2           -0.1964           -1.6336j            1.6454           -0.2690
AR.3           -0.1964           +1.6336j            1.6454            0.2690
AR.4           -1.9081           -0.0000j            1.9081           -0.5000
-----------------------------------------------------------------------------
"""

In [266]:
model2 = AutoReg(sdf['mid_price'], lags=4)
model2_fitted = model2.fit()
model2_fitted.summary()

c:\Users\giova\Git Repositories\Prosperity\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:              mid_price   No. Observations:                 2000
Model:                     AutoReg(4)   Log Likelihood               -3467.787
Method:               Conditional MLE   S.D. of innovations              1.375
Date:                Wed, 27 Mar 2024   AIC                           6947.574
Time:                        21:00:45   BIC                           6981.167
Sample:                             4   HQIC                          6959.910
                                 2000                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            4.3920     10.797      0.407      0.684     -16.769      25.553
mid_price.L1     0.3304      0.022     15.050      0.000       0.287       0.373
mid_price.L2     0.2235      0.022      9.943      0.000       0.179       0.268
mid_price.L3     0.2517      0.022     11.196      0.000       0.208       0.296
mid_price.L4     0.1935      0.022      8.816      0.000       0.150       0.237
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.0004           -0.0000j            1.0004           -0.0000
AR.2           -0.1964           -1.6336j            1.6454           -0.2690
AR.3           -0.1964           +1.6336j            1.6454            0.2690
AR.4           -1.9081           -0.0000j            1.9081           -0.5000
-----------------------------------------------------------------------------
"""

In [267]:
model_lst = []
for i in range(1, 6):
    model = AutoReg(sdf['mid_price'], lags=i)
    model_fitted = model.fit()
    model_lst.append(model_fitted)

c:\Users\giova\Git Repositories\Prosperity\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\giova\Git Repositories\Prosperity\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\giova\Git Repositories\Prosperity\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\giova\Git Repositories\Prosperity\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\giova\Git Repositories\Prosperity\.venv\Lib\site-packages\statsmodels\tsa\b

In [268]:
def calc_vals(model_lst):
    i = 1
    v_dict = {}
    for model in model_lst:
        p_arr = np.array(model.params[1:])

        l_arr = np.array([sdf['mid_price'].shift(i) for i in range(len(p_arr))])

        p_arr_reshaped = p_arr.reshape(-1, 1)

        s = p_arr_reshaped * l_arr
        s = np.sum(s, axis=0)

        r = s - sdf['mid_price'] + model.params[0]

        v_dict[i] = (r.mean(), r.std())
        i += 1
    
    return v_dict

In [269]:
d = calc_vals(model_lst)

C:\Users\giova\AppData\Local\Temp\ipykernel_27056\555390493.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  r = s - sdf['mid_price'] + model.params[0]


In [270]:
d

{1: (-0.02517120555356132, 0.09215347109240035),
 2: (-0.02387275809685233, 0.761006967325334),
 3: (-0.023873438207679405, 0.9028859262739437),
 4: (-0.023633226611347084, 0.9433399546852195),
 5: (-0.02314794128332841, 0.9651057967178366)}

In [271]:
model = AutoReg(sdf['mid_price'], lags=4)
model_fitted = model.fit()
model_fitted.summary()

c:\Users\giova\Git Repositories\Prosperity\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:              mid_price   No. Observations:                 2000
Model:                     AutoReg(4)   Log Likelihood               -3467.787
Method:               Conditional MLE   S.D. of innovations              1.375
Date:                Wed, 27 Mar 2024   AIC                           6947.574
Time:                        21:00:48   BIC                           6981.167
Sample:                             4   HQIC                          6959.910
                                 2000                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            4.3920     10.797      0.407      0.684     -16.769      25.553
mid_price.L1     0.3304      0.022     15.050      0.000       0.287       0.373
mid_price.L2     0.2235      0.022      9.943      0.000       0.179       0.268
mid_price.L3     0.2517      0.022     11.196      0.000       0.208       0.296
mid_price.L4     0.1935      0.022      8.816      0.000       0.150       0.237
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.0004           -0.0000j            1.0004           -0.0000
AR.2           -0.1964           -1.6336j            1.6454           -0.2690
AR.3           -0.1964           +1.6336j            1.6454            0.2690
AR.4           -1.9081           -0.0000j            1.9081           -0.5000
-----------------------------------------------------------------------------
"""

In [272]:
val4 = sdf['mid_price'].shift(3)
val3 = sdf['mid_price'].shift(2)
val2 = sdf['mid_price'].shift(1)
val1 = sdf['mid_price'].shift(0)

co4, co3, co2, co1 = [-0.01869561,  0.0455032,  0.16316049,  0.8090892]
a = 4.481696494462085

vals = co4*val4 +  co3*val3  +co2*val2 + co1*val1
s = vals - sdf['mid_price'] + a
s.mean(), s.std()

(-0.20856933907600197, 0.3056399144999105)